In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np
import pickle
import os

In [10]:
with open("data/safe_data_set.pkl", "rb") as f:
    safe_data = pickle.load(f)  # Load the Pickle file

# Convert to PyTorch tensor
safe_data = torch.tensor(safe_data, dtype=torch.float32)
# Use safe data in training
states_safe = safe_data  # Use this in your training loop

with open("data/unsafe_data_set.pkl", "rb") as f2:
    unsafe_data = pickle.load(f2)  # Load the Pickle file

unsafe_data = torch.tensor(unsafe_data, dtype=torch.float32)
states_unsafe = unsafe_data
next_states_safe = states_safe[1:]  # All but the first state
states_safe = states_safe[:-1]  # All but the last state
print(states_safe[0])
print(next_states_safe[0])
print(len(states_safe))


tensor([ 397.6137,  852.4254, -122.5904,   10.0000])
tensor([ 393.7723,  861.6581, -112.5905,   10.0000])
19999


In [ ]:
class BarrierFunctionNet(nn.Module):
    def __init__(self, input_dim):
        super(BarrierFunctionNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        hidden = self.tanh(self.fc1(x))
        return self.fc2(hidden)

In [4]:
def phi(x):
    """ReLU function to enforce barrier constraints."""
    return torch.maximum(x, torch.tensor(0.0))

def barrier_loss(B_s, B_u, Lf_B_s, gamma, ws=1.0, wu=1.0, wl=1.0):
    """
    Computes the barrier function loss L(θ).

    Args:
        B_s: Barrier function output for safe states (Tensor of shape [Ns]).
        B_u: Barrier function output for unsafe states (Tensor of shape [Nu]).
        Lf_B_s: Lie derivative of B(x) for safe states (Tensor of shape [Ns]).
        gamma: Weight for the Lie derivative constraint.
        ws, wu, wl: Weights for different loss terms.

    Returns:
        Total loss value.
    """
    Ns = B_s.shape[0]  # Number of safe samples
    Nu = B_u.shape[0]  # Number of unsafe samples

    # Compute each term of the loss
    loss_safety = ws * torch.mean(phi(-B_s))   # First term
    loss_usability = wu * torch.mean(phi(B_u)) # Second term
    loss_invariance = wl * torch.mean(phi(-Lf_B_s - gamma * B_s))  # Third term

    # Combine the terms
    total_loss = loss_safety + loss_usability + loss_invariance
    return total_loss

In [6]:
def compute_Lf_B_approx(barrier_net, states, next_states, delta_t):
    """
    Approximates the Lie derivative Lf B(x) using sampled trajectories.

    Args:
        barrier_net: Neural network for the barrier function.
        states: Current states (Tensor of shape [Ns, state_dim]).
        next_states: Next states (Tensor of shape [Ns, state_dim]).
        delta_t: Time difference between consecutive states.

    Returns:
        Approximation of Lf B(x).
    """
    # Compute B(s) and B(s0)
    B_s = barrier_net(states)
    B_next = barrier_net(next_states)

    # Approximation: (B(s') - B(s)) / delta_t
    Lf_B = (B_next - B_s) / delta_t
    Lf_B_scalar = Lf_B.mean()
    return Lf_B_scalar

In [ ]:
# Load the saved checkpoint
checkpoint = torch.load('checkpoint9.pth')

# Reinitialize the model and optimizer
barrier_net = BarrierFunctionNet(4)
optimizer = Adam(barrier_net.parameters(), lr=1e-3)

# Load the state_dict for the model and optimizer
barrier_net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Get the last epoch and loss if needed
start_epoch = checkpoint['epoch']
last_loss = checkpoint['loss']

# Set the model to evaluation mode or training mode
barrier_net.train()  # or barrier_net.eval() if you want to evaluate

C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\495214539.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint6.pth')


BarrierFunctionNet(
  (fc1): Linear(in_features=4, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (tanh): Tanh()
)

In [ ]:
# Hyperparameters
input_dim = 4  # Example input dimension for state
gamma = 1    # Weight for invariance term
ws, wu, wl = 1, 1, 1  # Loss weights
delta_t = 0.1  # Time step

# Initialize the barrier function network and optimizer
barrier_net = BarrierFunctionNet(input_dim)
optimizer = Adam(barrier_net.parameters(), lr=1e-3)

# Training data (replace with actual safe/unsafe state data)
Ns, Nu = 20000, 20000  # Number of safe and unsafe samples


# Initialize the barrier function network and optimizer
barrier_net = BarrierFunctionNet(input_dim)
optimizer = Adam(barrier_net.parameters(), lr=1e-3)

# Check if a checkpoint exists and load it
checkpoint_path = 'checkpoint9.pth'
start_epoch = 1  # default start epoch if no checkpoint is found
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    barrier_net.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1  # Continue from the next epoch
    print(f"Resuming training from epoch {start_epoch} with loss: {checkpoint['loss']}")

# Training loop (starting from the right epoch)
for epoch in range(start_epoch, 2000):
    # Forward pass: Compute B(x) for safe and unsafe states
    B_s = barrier_net(states_safe)
    B_u = barrier_net(states_unsafe)

    # Compute Lf B(x) approximation for safe states
    Lf_B_s = compute_Lf_B_approx(barrier_net, states_safe, next_states_safe, delta_t)

    # Compute the loss
    loss = barrier_loss(B_s, B_u, Lf_B_s, gamma, ws, wu, wl)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

    # Save model and optimizer state every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,  # current epoch
            'model_state_dict': barrier_net.state_dict(),  # model parameters
            'optimizer_state_dict': optimizer.state_dict(),  # optimizer parameters
            'loss': loss.item(),  # last loss value
        }, checkpoint_path)


Resuming training from epoch 139350 with loss: 0.0014598967973142862


C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\324380667.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [28]:
def verify_safe_states(barrier_net, states_safe):
    margin = 300  # Allowable violations
    barrier_values = barrier_net(states_safe)  # Compute B(x) for all safe states
    num_violations = torch.sum(barrier_values < 0).item()  # Count violations
    
    if num_violations > margin:
        print(f"Condition 1 violated: {num_violations} states have B(x) < 0, exceeding the margin of {margin}")
        return False
    else:
        print(f"Condition 1 satisfied: {num_violations} violations within the margin of {margin}")
        return True


In [29]:
def verify_unsafe_states(barrier_net, states_unsafe):
    margin = 300  # Allowable violations
    barrier_values = barrier_net(states_unsafe)  # Compute B(x) for all unsafe states
    num_violations = torch.sum(barrier_values > 0).item()  # Count violations
    if num_violations > margin:
        print(f"Condition 1 violated: {num_violations} states have B(x) > 0, exceeding the margin of {margin}")
        return False
    else:
        print(f"Condition 1 satisfied: {num_violations} violations within the margin of {margin}")
        return True


In [11]:
def create_grid_around_boundary(states, threshold=0.1, delta=0.05):
    # Create a grid around the boundary where B(x) = 0, checking zero crossing
    grid_states = []
    
    # Loop over each state in the list of states
    for state in states:
        # Convert state to a tensor if it's not already
        state_tensor = torch.tensor(state, dtype=torch.float32)
        
        # Compute the barrier function for this state
        barrier_value = barrier_net(state_tensor).mean().item()  # Take the mean of the output vector
        
        # If B(x) is near 0 (within threshold), create a grid around this state
        if np.abs(barrier_value) < threshold:  
            # Create a grid around this state within a range of delta
            for dx in np.linspace(-delta, delta, num=5):
                # Apply dx element-wise to each dimension of the state
                grid_state = state + np.array([dx] * len(state))  # Add dx to each dimension of state
                grid_states.append(grid_state.tolist())  # Ensure grid_state is a list
                
    # Convert grid states to a tensor and return
    return torch.tensor(grid_states, dtype=torch.float32)



In [ ]:
def compute_lie_derivative(barrier_net, state, dynamics_model, control_input=None):
    if control_input is None:
        control_input = np.array([0.0, 0.0])  # Default control input: [acceleration, steering] = [1, 0]
    
    state_tensor = torch.tensor(state, dtype=torch.float32, requires_grad=True)
    
    # Compute barrier function value B(x)
    B_x = barrier_net(state_tensor)
    
    # Ensure B_x is scalar
    B_x = B_x.mean() 
    
    # Compute gradient of B(x) w.r.t. state
    B_x.backward()
    grad_B_x = state_tensor.grad  # ∇B(x)

    f_x = torch.tensor(dynamics_model(state_tensor, control_input), dtype=torch.float32)

    # Compute Lie derivative Lf B(x) = ∇B(x) ⋅ f(x)
    Lf_B = torch.dot(grad_B_x, f_x).item()  # Use .item() to extract the scalar value

    return Lf_B


In [13]:
def unique_tensor_rows(tensor):
    """Remove duplicate rows in a PyTorch tensor"""
    return torch.unique(tensor, dim=0)

In [21]:
def verify_lie_derivative(barrier_net, grid_states, dynamics_model, delta_t=0.1, B_U_threshold=0.01, B_L_threshold=-0.01):
    default_control_input = np.array([0.0, 0.0])  # Example: [acceleration, steering] = [0, 0]
    violating_states = []
    
    for state in grid_states:
        # Ensure state is a numpy array or a list
        state_tensor = torch.tensor(state, dtype=torch.float32)
        
        # Compute the barrier function B(x) and its Lie derivative Lf B(x)
        B_x = barrier_net(state_tensor).mean().item()  # Barrier function value
        Lf_B = compute_lie_derivative(barrier_net, state_tensor, dynamics_model, delta_t, control_input=default_control_input)
        
        # Check if the conditions for violation are met
        B_U = max(B_x, 0)  # Assuming B_U(x) corresponds to max(B(x), 0)
        B_L = min(B_x, 0)  # Assuming B_L(x) corresponds to min(B(x), 0)
        
        # All three conditions must be satisfied for a state to be a violation
        if B_U > B_U_threshold and B_L < B_L_threshold and Lf_B <= 0:
            violating_states.append(state)

    print(f"Condition 3 violated: {len(violating_states)} states found.")
    
    return violating_states


In [15]:
def dynamics_model(state, control_input):
    if isinstance(state, torch.Tensor):
        state_values = state.detach().numpy()  # Convert the tensor to a numpy array
    else:
        state_values = state  # If it's already a numpy array, use it directly
    
    # Unpack the state values
    x, y, theta, v = state_values[0], state_values[1], state_values[2], state_values[3]
    a, delta = control_input  # Extract control inputs (acceleration, steering angle)
    #print(x,y)
    L = 2.5  # Wheelbase length (for example)
    
    # Compute the next state using the dynamics equations
    dx = v * np.cos(theta) # Change in x position
    dy = v * np.sin(theta) # Change in y position
    delta = (v / L) * np.tan(delta) # Change in heading (yaw rate)
    dv = a  # Change in velocity (acceleration)
    #print(f"dx: {dx}, dy: {dy}, dtheta: {dtheta}, dv: {dv}")
    return np.array([dx, dy, delta, dv], dtype=float)  # Explicitly set the dtype to ensure consistent types



In [19]:

# Condition 1: Verify safe states
verify_safe_states(barrier_net, states_safe)

# Condition 2: Verify unsafe states
verify_unsafe_states(barrier_net, states_unsafe)

# Condition 3: Verify Lie derivative for boundary states
#does not work when using the create_grid_around_boundary2 function
grid_states = create_grid_around_boundary(states_safe, threshold=0.5, delta=0.1)
#grid_states = create_grid_around_boundary(states_safe, threshold=0.1, delta=0.05)
violating_states = verify_lie_derivative(barrier_net, grid_states, dynamics_model)


Condition 1 satisfied: B(x) >= 0 for all x in X_s
Condition 2 violated: B(x) is non-negative for some x in X_u


C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\3441715354.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state, dtype=torch.float32)
C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\3441715354.py:18: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  grid_state = state + np.array([dx] * len(state))  # Add dx to each dimension of state
C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\406496864.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state, dtype=torch.float32)
C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\1941408205.py:5: 

Condition 3 violated: 0 states found.


In [16]:
from sklearn.neighbors import KNeighborsClassifier
def knn_labeling(states, labels, test_states, k=15):
    # Fit kNN classifier on the states with known labels
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(states, labels)
    
    # Predict labels for the test states
    predicted_labels = knn.predict(test_states)
    
    return predicted_labels

In [20]:

# Prepare training data (safe and unsafe states)
training_states = np.vstack([states_safe, states_unsafe])  # Stack them together
training_labels = np.array([1] * len(states_safe) + [0] * len(states_unsafe))  # Safe = 1, Unsafe = 0

# Convert violating states into a NumPy array
violating_states_np = np.array([state.detach().numpy() for state in violating_states])

# Use kNN to predict the labels for the violating states
violating_labels = knn_labeling(training_states, training_labels, violating_states_np)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [18]:
num_safe_violations = (violating_labels == 1).sum().item()  # Count where label is 1
num_unsafe_violations = (violating_labels == 0).sum().item()  # Count where label is 0

print(f"Safe violating states: {num_safe_violations}")
print(f"Unsafe violating states: {num_unsafe_violations}")


NameError: name 'violating_labels' is not defined

In [ ]:
cond1 = verify_safe_states(barrier_net, states_safe)
cond2 =  verify_unsafe_states(barrier_net, states_unsafe)
safe_violating_states = []
unsafe_violating_states = []
checkpoint_path = 'checkpoint9.pth'
checkpoint_path_save = 'checkpoint9.pth'

for i in range(200):
    count = 0
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        barrier_net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # Continue from the next epoch
        print(f"Resuming training from epoch {start_epoch} with loss: {checkpoint['loss']}")
    for epoch in range(start_epoch, start_epoch+100):
        # Forward pass: Compute B(x) for safe and unsafe states
        B_s = barrier_net(states_safe)
        B_u = barrier_net(states_unsafe)

        # Compute Lf B(x) approximation for safe states
        Lf_B_s = compute_Lf_B_approx(barrier_net, states_safe, next_states_safe, delta_t)

        # Compute the loss
        loss = barrier_loss(B_s, B_u, Lf_B_s, gamma, ws, wu, wl)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

        # Save model and optimizer state every 10 epochs
        if (epoch + 1) % 10 == 0:
            torch.save({
                'epoch': epoch,  # current epoch
                'model_state_dict': barrier_net.state_dict(),  # model parameters
                'optimizer_state_dict': optimizer.state_dict(),  # optimizer parameters
                'loss': loss.item(),  # last loss value
            }, checkpoint_path_save)
        
    cond1 = verify_safe_states(barrier_net, states_safe)

    # Condition 2: Verify unsafe states
    cond2 = verify_unsafe_states(barrier_net, states_unsafe)

    # Condition 3: Verify Lie derivative for boundary states
    grid_states = create_grid_around_boundary(states_safe, threshold=0.1, delta=0.05)

    violating_states = verify_lie_derivative(barrier_net, grid_states, dynamics_model)
    violating_states_np = np.array([state.detach().numpy() for state in violating_states])
    if violating_states_np.size > 0:
        violating_labels = knn_labeling(training_states, training_labels, violating_states_np)

        # Split based on predicted labels
        safe_violating_states = violating_states_np[violating_labels == 1]
        unsafe_violating_states = violating_states_np[violating_labels == 0]


    # Update training sets
    if len(safe_violating_states) > 0:
        safe_violating_states = torch.tensor(safe_violating_states, dtype=torch.float32)
        states_safe = torch.cat([states_safe, safe_violating_states], dim=0)
        states_safe = unique_tensor_rows(states_safe)
        next_states_safe = states_safe[1:]  # All but the first state
        states_safe = states_safe[:-1]  # All but the last state

    if len(unsafe_violating_states) > 0:
        unsafe_violating_states = torch.tensor(unsafe_violating_states, dtype=torch.float32)
        states_unsafe = torch.cat([states_unsafe, unsafe_violating_states], dim=0)
        states_unsafe = unique_tensor_rows(states_unsafe)
    else:
        print("no violating states")
    if cond1:
        count+=1
    if cond2:
        count+=1
    if violating_states_np.size < 60:
        count+=1
    if count > 2 :
        print("goood enough")
        break
        
    # Print out the labels for violating states
    # for i, state in enumerate(violating_states):
    #     print(f"State {state}: Label {violating_labels[i]}")


Condition 1 satisfied: 146 violations within the margin of 300
Condition 1 violated: 967 states have B(x) > 0, exceeding the margin of 300
Resuming training from epoch 164550 with loss: 0.00127696234267205


C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\1366368615.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Epoch 164560, Loss: 0.0008
Epoch 164570, Loss: 0.0008
Epoch 164580, Loss: 0.0006
Epoch 164590, Loss: 0.0006
Epoch 164600, Loss: 0.0006
Epoch 164610, Loss: 0.0005
Epoch 164620, Loss: 0.0007
Epoch 164630, Loss: 0.0005
Epoch 164640, Loss: 0.0006
Epoch 164650, Loss: 0.0008
Condition 1 violated: 8165 states have B(x) < 0, exceeding the margin of 300
Condition 1 satisfied: 8 violations within the margin of 300


C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\3441715354.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state, dtype=torch.float32)
C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\3441715354.py:18: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  grid_state = state + np.array([dx] * len(state))  # Add dx to each dimension of state
C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\406496864.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state_tensor = torch.tensor(state, dtype=torch.float32)
C:\Users\Zhenyu\AppData\Local\Temp\ipykernel_3540\1941408205.py:5: 

Condition 3 violated: 0 states found.
no violating states
Resuming training from epoch 164650 with loss: 0.0008123196312226355
Epoch 164660, Loss: 0.0076
Epoch 164670, Loss: 0.0063
Epoch 164680, Loss: 0.0022
Epoch 164690, Loss: 0.0013
Epoch 164700, Loss: 0.0009
Epoch 164710, Loss: 0.0008
Epoch 164720, Loss: 0.0007
Epoch 164730, Loss: 0.0006
Epoch 164740, Loss: 0.0005
Epoch 164750, Loss: 0.0006
Condition 1 satisfied: 85 violations within the margin of 300
Condition 1 violated: 473 states have B(x) > 0, exceeding the margin of 300
Condition 3 violated: 0 states found.
no violating states
Resuming training from epoch 164750 with loss: 0.0005775589379481971
Epoch 164760, Loss: 0.0005
Epoch 164770, Loss: 0.0006
Epoch 164780, Loss: 0.0004
Epoch 164790, Loss: 0.0005
Epoch 164800, Loss: 0.0006
Epoch 164810, Loss: 0.0004
Epoch 164820, Loss: 0.0006
Epoch 164830, Loss: 0.0004
Epoch 164840, Loss: 0.0006
Epoch 164850, Loss: 0.0011
Condition 1 satisfied: 9 violations within the margin of 300
Conditi